In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import Libraries


Lesgoooo


In [ ]:
import struct
import numpy as np
import tensorflow as tf
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Flatten,Dense,Dropout
from keras.layers import UpSampling2D,Activation,LeakyReLU,MaxPooling2D,Normalization,SeparableConv2D
from keras.layers.merge import add, concatenate
from keras.models import Model,Sequential
import keras.losses
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing import image as im

Paths

In [ ]:
test ='/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test'
train= '/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train'

Real and Augmented Data 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


new = ImageDataGenerator(rescale = 1.0/255.0,
                         horizontal_flip = True,
                         zoom_range =.2,
                         shear_range = 0.2,
                         width_shift_range=0.01, 
                         height_shift_range=0.01 
                              
                         )

train = new.flow_from_directory(train,target_size=(224,224),class_mode='binary',color_mode='grayscale',batch_size=32)
valid = ImageDataGenerator(rescale = 1.0/255.0).flow_from_directory(test,target_size=(224,224),class_mode='binary',color_mode='grayscale',batch_size=32)


Model ------> Like *VGG16* BUT **slightly** change

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), kernel_initializer = 'he_uniform',activation='LeakyReLU', input_shape=(224, 224, 1),padding='same'))
model.add(Conv2D(64,kernel_size=(3,3),strides=(2,2),kernel_initializer = 'he_uniform',activation='LeakyReLU',padding='same'))
model.add(MaxPooling2D(strides=(2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))
model.add(Conv2D(128,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))
model.add(MaxPooling2D(strides=(2,2)))
model.add(Conv2D(256,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))
model.add(Conv2D(256,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))
model.add(Conv2D(256,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))

model.add(MaxPooling2D(strides=(2,2)))
model.add(Conv2D(512,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))
model.add(Conv2D(512,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))
model.add(Conv2D(512,kernel_size=(3,3),activation='LeakyReLU',padding='same',kernel_initializer = 'he_uniform',))

model.add(MaxPooling2D(strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096,activation='LeakyReLU'))
model.add(Dense(4096,activation='LeakyReLU'))
model.add(Dense(1,activation='sigmoid'))


from tensorflow.keras.optimizers import Adam,RMSprop,SGD
model.compile(optimizer = SGD(.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

It's time to fit

In [ ]:
model.fit(train,validation_data=valid,epochs=5,batch_size =5)

### Evalute model


In [ ]:
val = model.evaluate(valid,steps=200)
print(val[1])

### Second model (Using depthwiseConvolution Layer)

In [ ]:
model = Sequential()

model.add(SeparableConv2D(64,kernel_size = 3,depthwise_initializer = 'he_uniform',activation='LeakyReLU', input_shape=(224, 224, 1),padding='same'))
model.add(MaxPooling2D(strides=(2,2)))
model.add(SeparableConv2D(128,kernel_size= 3,activation='LeakyReLU',padding='same',depthwise_initializer = 'he_uniform',))
model.add(MaxPooling2D(strides=(2,2)))
model.add(SeparableConv2D(256,kernel_size= 3,activation='LeakyReLU',padding='same',depthwise_initializer = 'he_uniform',))

model.add(MaxPooling2D(strides=(2,2)))
model.add(SeparableConv2D(512,kernel_size= 3,activation='LeakyReLU',padding='same',depthwise_initializer = 'he_uniform',))

model.add(MaxPooling2D(strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096,activation='LeakyReLU'))
model.add(Dense(4096,activation='LeakyReLU'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

from tensorflow.keras.optimizers import Adam,RMSprop,SGD
model.compile(optimizer = SGD(.01),loss='binary_crossentropy',metrics=['accuracy'])

### Train model 

In [ ]:
model.fit(train,validation_data=valid,epochs=10,batch_size=32)

### Amazing thing just happend here .. the first model took 9-8/ steps

### But 2nd model took 3s/step ...  which is faster surprisingly.. at least to me -_- 

Evaluate model

In [ ]:
val = model.evaluate(valid,steps=200)
print(val[1])

In [ ]:
for i in os.listdir(os.path.join(test,'PNEUMONIA')):
  locP = im.load_img(test+'/PNEUMONIA/'+i, target_size = (224,224), color_mode = 'grayscale')
  imagePneu = im.img_to_array(locP)
  imagePneu = tf.expand_dims(imagePneu, axis =0 )
  predPneumonia =model.predict(imagePneu) 
for i in os.listdir(os.path.join(test,'NORMAL')):
  locN = im.load_img(test+'/NORMAL/'+i, target_size = (224,224), color_mode = 'grayscale')
  imageNor = im.img_to_array(locN)
  imageNor = tf.expand_dims(imageNor, axis =0 )
  predNormal =model.predict(imageNor)

In [ ]:
wrongPneu=[]
for i in range(len(predPneumonia)):
  if predPneumonia != 1:
    wrongPneu.append(i)

wrongNor=[]
for i in range(len(predNormal)):
  if predNormal != 0:
    wrongNor.append(i)
    
print(len(wrongPneu),len(wrongNor))

**VISUALIZATION***

In [ ]:
locP = []
locN = []
for i in os.listdir(os.path.join(test,'PNEUMONIA')):
  locP.append(im.load_img(test+'/PNEUMONIA/'+i, target_size = (224,224)))
for i in os.listdir(os.path.join(test,'NORMAL')):
  locN.append(im.load_img(test+'/NORMAL/'+i, target_size = (224,224)))


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
for i in enumerate(locN):
  plt.subplot(4,5,i[0]+1)
  plt.imshow(i[1])
  plt.xlabel('NORMAL')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
for i in enumerate(locP):
  plt.subplot(4,5,i[0]+1)
  plt.imshow(i[1])
  plt.xlabel('PNEUMONIA')